In [11]:
import pandas as pd
df = pd.read_csv('data.csv', parse_dates=['Transaction Date'])
#Clear non needed types
df = df[~df['Type'].isin(['Municipal Security', 'Other Securities','Non-Public Stock', 'Corporate Bond'])]
df

,Stock,Transaction Date,Buy/Sell,Type,Amount,Senator
64,RGC,2017-03-28,Sale,Stock,"$500,001 - $1,000,000",Michael f Bennet
65,RGC,2017-12-08,Sale,Stock,"$500,001 - $1,000,000",Michael f Bennet
66,LVLT\nCTL,2017-11-01,Exchange,Stock,"$50,001 - $100,000",Michael f Bennet
70,KHC,2019-11-26,Sale,Stock Option,"$1,001 - $15,000",Roy Blunt
71,MDLZ,2019-12-02,Sale,Stock Option,"$50,001 - $100,000",Roy Blunt
...,...,...,...,...,...,...
15800,NFLX,2020-09-14,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15801,NFLX,2020-09-14,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15802,NFLX,2020-09-23,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15803,AAPL,2022-02-24,Sale,Stock,"$50,001 - $100,000",Ron l Wyden


In [20]:
#Drop exchanges
df = df[df['Buy/Sell'] != 'Exchange']
df

,Stock,Transaction Date,Buy/Sell,Type,Amount,Senator
64,RGC,2017-03-28,Sale,Stock,"$500,001 - $1,000,000",Michael f Bennet
65,RGC,2017-12-08,Sale,Stock,"$500,001 - $1,000,000",Michael f Bennet
70,KHC,2019-11-26,Sale,Stock Option,"$1,001 - $15,000",Roy Blunt
71,MDLZ,2019-12-02,Sale,Stock Option,"$50,001 - $100,000",Roy Blunt
72,CSCO,2015-12-11,Sale,Stock,"$1,001 - $15,000",Roy Blunt
...,...,...,...,...,...,...
15800,NFLX,2020-09-14,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15801,NFLX,2020-09-14,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15802,NFLX,2020-09-23,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15803,AAPL,2022-02-24,Sale,Stock,"$50,001 - $100,000",Ron l Wyden


In [27]:
#Make NaNs in Type == Stock
df['Type'].fillna('Stock', inplace=True)

,Stock,Transaction Date,Buy/Sell,Type,Amount,Senator
64,RGC,2017-03-28,Sale,Stock,"$500,001 - $1,000,000",Michael f Bennet
65,RGC,2017-12-08,Sale,Stock,"$500,001 - $1,000,000",Michael f Bennet
70,KHC,2019-11-26,Sale,Stock Option,"$1,001 - $15,000",Roy Blunt
71,MDLZ,2019-12-02,Sale,Stock Option,"$50,001 - $100,000",Roy Blunt
72,CSCO,2015-12-11,Sale,Stock,"$1,001 - $15,000",Roy Blunt
...,...,...,...,...,...,...
15800,NFLX,2020-09-14,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15801,NFLX,2020-09-14,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15802,NFLX,2020-09-23,Sale,Stock,"$1,001 - $15,000",Ron l Wyden
15803,AAPL,2022-02-24,Sale,Stock,"$50,001 - $100,000",Ron l Wyden


In [28]:
#irregular stocks to fix
irregular = df[df['Stock'].apply(lambda x: len(x) > 5)]
irregular

,Stock,Transaction Date,Buy/Sell,Type,Amount,Senator
74,MDLZ (stock option)\nOption Type: Neither\nStr...,2015-12-30,Sale,Stock Option,"$50,001 - $100,000",Roy Blunt
81,MDLZ (stock option),2014-12-30,Sale,Stock,"$50,001 - $100,000",Roy Blunt
85,The Kraft Heinz Company - Common Stock\nOption...,2022-12-23,Sale,Stock Option,"$1,001 - $15,000",Roy Blunt
160,AMGEN INC (AMGN),2015-11-23,Purchase,Stock,"$1,001 - $15,000",Shelley m Capito
165,ETN-Eaton Corporation plc,2017-10-25,Sale,Stock,"$1,001 - $15,000",Shelley m Capito
...,...,...,...,...,...,...
15721,NASSAU CNTY N Y SWR & STORM WTR FIN AUT REV BD...,2014-01-03,Purchase,Stock,"$15,001 - $50,000",Ron Wyden
15722,ERIE CNTY NY INDL DEV AGY SCH FAC REV REV BDS ...,2014-01-03,Purchase,Stock,"$15,001 - $50,000",Ron Wyden
15723,new york st dorm auth lease bond fund,2013-08-07,Purchase,Stock,"$100,001 - $250,000",Ron Wyden
15724,triborough brdg & Tunl auth ny rev bond,2013-09-30,Purchase,Stock,"$15,001 - $50,000",Ron Wyden
